In [33]:
from volumeSpikeDetector import *
from utility import *
# volume spike detector
import pandas as pd
import numpy as np
from datetime import date
from nsepy import symbols
from indicators import *
import yfinance as yf
import sys
from IPython.display import display, clear_output
from IPython.utils import io
import time



In [38]:
with io.capture_output() as captured:
    loaded_data=load_data()
    isDataLoaded=True

In [35]:
with io.capture_output() as captured:

    result=filterVolumeSpikeWithAverage(data,2)
    result['upOrDown']=np.where((result["Close"]-result["Open"]>0)>0,1,0)
    table=result.loc['2023-07-19'].reset_index(drop=True).style.apply(highlight_greaterthan, threshold=1, column='upOrDown', axis=1)

In [36]:
table

,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA200,upOrDown
0,290.700012,296.750000,289.350006,293.450012,293.450012,4716085.000000,CROMPTON,318.229250,1
1,356.899994,379.200012,356.000000,373.600006,373.600006,3455108.000000,KRBL,379.170500,1
2,65.349998,68.500000,58.049999,59.500000,59.500000,22333111.000000,NETWORK18,62.231000,0
3,2209.899902,2258.199951,2197.550049,2238.850098,2238.850098,63645.000000,VMART,2494.866501,1
4,770.099976,792.849976,764.099976,777.849976,777.849976,5316153.000000,VOLTAS,828.304750,1


In [11]:
yf.download("ITC.NS",start="2023-07-14",end="2023-07-14", interval='1m')

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ITC.NS: No data found for this date range, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [ ]:
from nsepy import symbols
from utility import * 

symbols=pd.DataFrame(symbols.get_index_constituents_list("nifty100"))
df_grouped=data.groupby("Symbol")
result=[]

for i in symbols.Symbol:
    try:
        temp=df_grouped.get_group(i)
        int_value=getIntrinsicValue(i)
        #temp=rsi(temp,4)
        #temp=sma(temp,200)
        #current_average=list(temp.sma200)[-1]
        current_price=list(temp.Close)[-1]
        current_rsi=list(temp.rsi)[-1];
        if current_rsi<30 and int_value<current_price:
            
            print(i,int_value,current_price)
    except:
        pass
        print(f"cannot fetch {i}")



In [37]:
import pandas as pd
import yfinance as yf


def filterVolumeSpikeWithAverage(data, percentage):
    result = pd.DataFrame()

    try:
        data_grouped = data.groupby(by="Symbol")
    except:
        print("Error in grouping")

    for symbol in data_grouped.groups:
        try:
            temp = data_grouped.get_group(symbol)
            temp_avg_vol = temp['Volume'].rolling(5).mean()
            temp['SMA200'] = temp['Close'].rolling(200).mean()

            temp = temp[(temp['Volume'] > percentage * temp_avg_vol) & (temp['Close'] < temp['SMA200'])]
            result = pd.concat([result, temp])
        except:
            print(f"Error in fetching {symbol}")

    return result

def apply_trailing_stop_loss(data, stop_loss_percentage=1):
    exit_prices = {}
    for symbol in data['Symbol'].unique():
        symbol_data = data[data['Symbol'] == symbol]
        entry_price = symbol_data.iloc[0]['Close']
        exit_prices[symbol] = entry_price * (1 - stop_loss_percentage / 100)

    exit_data = pd.DataFrame({'Symbol': list(exit_prices.keys()), 'ExitPrice': list(exit_prices.values())})
    data = data.merge(exit_data, on='Symbol', how='left')
    data['Exit'] = data['Low'] <= data['ExitPrice']
    return data

# Main part of the strategy
# Get historical price data for your desired stocks (e.g., Nifty 100 index constituents)
# For example, if you have a list of symbols, you can do:
# symbols = ['AAPL', 'GOOGL', 'MSFT', ...]
# data = get_historical_data_for_symbols(symbols)

# Here, we'll use the `filterVolumeSpikeWithAverage` function you provided to get filtered data.
data = filterVolumeSpikeWithAverage(loaded_data, percentage=2)

# Apply the trailing stop loss exit condition
data_with_exit = apply_trailing_stop_loss(data)

# You can now analyze the data and evaluate the performance of your strategy.
print(data_with_exit.head())




/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

...

/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['SMA200'] = temp['Close'].rolling(200).mean()
/tmp/ipykernel_15837/286333289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

           Open          High           Low         Close     Adj Close  \
0  22130.000000  22130.000000  21400.000000  21505.699219  21505.699219   
1  21401.000000  21675.000000  21300.449219  21602.000000  21602.000000   
2  22302.000000  22579.949219  21500.000000  21554.000000  21554.000000   
3  21949.949219  22439.949219  21600.000000  22091.150391  22091.150391   
4  22400.000000  22765.500000  22125.250000  22412.949219  22412.949219   

    Volume   Symbol        SMA200     ExitPrice   Exit  
0   4561.0  3MINDIA  21943.205703  21290.642227  False  
1  10545.0  3MINDIA  21953.403203  21290.642227  False  
2  48590.0  3MINDIA  22287.869453  21290.642227  False  
3  88819.0  3MINDIA  22386.963213  21290.642227  False  
4   6232.0  3MINDIA  22975.226250  21290.642227  False  


In [32]:
#performance evaluation
import pandas as pd
import numpy as np

# Assuming you have already defined the data_with_exit DataFrame

def calculate_profit_loss(data):
    data['P&L'] = np.where(data['Exit'], data['ExitPrice'] - data['Close'], 0)
    data['CumulativeP&L'] = data['P&L'].cumsum()
    return data

def calculate_metrics(data):
    total_trades = len(data)
    winning_trades = len(data[data['P&L'] > 0])
    win_rate = (winning_trades / total_trades) * 100
    average_return = data['P&L'].mean()
    max_drawdown = (data['CumulativeP&L'] - data['CumulativeP&L'].cummax()).min()
    sharpe_ratio = data['P&L'].mean() / data['P&L'].std() * np.sqrt(252)  # Assuming daily data (252 trading days in a year)

    metrics = {
        'TotalTrades': total_trades,
        'WinningTrades': winning_trades,
        'WinRate(%)': win_rate,
        'AverageReturnPerTrade': average_return,
        'MaxDrawdown': max_drawdown,
        'SharpeRatio': sharpe_ratio
    }
    return metrics

def plot_equity_curve(data):
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 6))
    plt.plot(data['Date'], data['CumulativeP&L'], label='Equity Curve', color='blue')
    plt.xlabel('Date')
    plt.ylabel('Cumulative P&L')
    plt.title('Equity Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

# Perform the analysis
data_with_exit = calculate_profit_loss(data_with_exit)
metrics = calculate_metrics(data_with_exit)

# Print the metrics
for key, value in metrics.items():
    print(f"{key}: {value}")

# Plot the equity curve
plot_equity_curve(data_with_exit)

TotalTrades: 1892
WinningTrades: 1087
WinRate(%): 57.45243128964059
AverageReturnPerTrade: 75.89364038558391
MaxDrawdown: -383.0464843749942
SharpeRatio: 4.892762590807512


KeyError: 'Date'

<Figure size 720x432 with 0 Axes>

In [51]:
watchLivePrice.reset_index(drop=True).style.apply(highlight_greaterthan, threshold=1, column='upOrDown', axis=1)


,Open,High,Low,Close,Adj Close,Volume,Symbol,upOrDown
0,1697.250000,1698.000000,1695.949951,1697.849976,1697.849976,1266,ZOMATO.NS,1
1,1740.650024,1740.650024,1740.650024,1740.650024,1740.650024,0,ZOMATO.NS,0
2,1031.449951,1031.449951,1031.449951,1031.449951,1031.449951,2755,ZOMATO.NS,0
3,1007.750000,1008.549988,1003.299988,1006.000000,1006.000000,35517,ZOMATO.NS,0
4,1130.000000,1131.699951,1129.000000,1129.349976,1129.349976,20842,ZOMATO.NS,0
5,91.650002,91.699997,91.599998,91.599998,91.599998,42809,ZOMATO.NS,0
6,878.000000,878.900024,876.250000,878.900024,878.900024,3229,ZOMATO.NS,1
7,139.000000,139.000000,138.500000,138.899994,138.899994,55546,ZOMATO.NS,0
8,1282.199951,1293.150024,1282.199951,1282.199951,1282.199951,1229566,ZOMATO.NS,0
9,2554.000000,2559.000000,2551.100098,2559.000000,2559.000000,16851,ZOMATO.NS,1


In [13]:
while(True):
    with io.capture_output() as captured:
        price=yf.download(tickers='INDUSINDBK.NS', period='1d', interval='1m',rounding=True).tail(1)
    clear_output(wait=True)
    display(price)
    time.sleep(0.3)
    if((price.Close<1066.4)[0]):
        AlertRing()
        break

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-03-31 10:13:00+05:30,1065.35,1065.35,1065.35,1065.35,1065.35,0


In [ ]:
def watchPrice(symbol,trigger,long=True):
    while(True):
    with io.capture_output() as captured:
        price=yf.download(tickers=symbol+'.NS', period='1d', interval='1m',rounding=True).tail(1)
    clear_output(wait=True)
    display(price)
    time.sleep(0.3)
    if((price.Close>369.9)[0] & long==True):
        AlertRing()
        break
    else if ((price.Close<trigger)[0] & long==False):
        AlertRing()
        break
    

In [ ]:
from random import uniform
import time
from IPython.display import display, clear_output

def black_box():
    i = 1
    while True:
        clear_output(wait=True)
        display('Iteration '+str(i)+' Score: '+str(uniform(0, 1)))
        time.sleep(1)
        i += 1


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-03-22,376.950012,378.0,374.200012,376.149994,376.149994,3360615


In [44]:
price=yf.download(tickers='NIFTY_FIN_SERVICE.NS',start="2023-02-25", end=date.today()).tail(1)

[*********************100%***********************]  1 of 1 completed


In [45]:
price

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-04-03,18096.650391,18185.150391,18041.75,18164.75,18164.75,0


## TextBlob for sentiment analysis

The TextBlob library in Python returns sentiment polarity and subjectivity values as part of its sentiment analysis.

* Polarity: Polarity represents the sentiment polarity of a text, ranging from -1 to 1. A polarity score of -1 indicates highly negative sentiment, 0 indicates neutral sentiment, and 1 indicates highly positive sentiment. Polarity values closer to -1 or 1 indicate stronger sentiment, while values closer to 0 indicate weaker or neutral sentiment.

* Subjectivity: Subjectivity represents the subjectivity of a text, ranging from 0 to 1. A subjectivity score of 0 indicates objective text, meaning it is factual or unbiased, while a score of 1 indicates highly subjective text, meaning it is opinionated or biased. Subjectivity values between 0 and 1 indicate varying degrees of subjectivity.

In [5]:
from newsapi import NewsApiClient
from textblob import TextBlob

# Set up your News API key
newsapi = NewsApiClient(api_key='4a51169c54fb4d2880c2f0445f1a2adc')

# Define the stock symbol for which you want to extract news
stock_symbol = 'GLENMARK'

# Define the number of news articles to retrieve
num_articles = 5

# Get news articles related to the stock symbol
articles = newsapi.get_everything(q=stock_symbol, language='en', sort_by='relevancy', page_size=num_articles)

# Extract sentiment from each news article
for article in articles['articles']:
    title = article['title']
    description = article['description']
    content = article['content']
    
    # Concatenate the title, description, and content of the article
    text = f"{title}. {description}. {content}"
    
    # Perform sentiment analysis using TextBlob
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    subjectivity = analysis.sentiment.subjectivity
    
    # Print the sentiment analysis results
    print("Title: ", title)
    print("Sentiment Polarity: ", polarity)
    print("Sentiment Subjectivity: ", subjectivity)
    print("----")


Title:  What should you do with Bandhan Bank, Kalyan and Glenmark Pharma?
Sentiment Polarity:  -0.016527777777777773
Sentiment Subjectivity:  0.2901388888888889
----
Title:  Topical Drug Delivery Global Market Report 2023
Sentiment Polarity:  -0.001785714285714285
Sentiment Subjectivity:  0.21857142857142856
----
Title:  Glenmark exploring stake sale in life sciences arm to reduce debt
Sentiment Polarity:  0.020833333333333332
Sentiment Subjectivity:  0.6666666666666666
----
Title:  Glenmark gets USFDA nod to market generic medication
Sentiment Polarity:  0.0
Sentiment Subjectivity:  0.0
----
Title:  Glenmark gets USFDA nod for generic drug with 180-day exclusivity
Sentiment Polarity:  0.0
Sentiment Subjectivity:  0.4
----


In [4]:
indices=["^NSEI","^NSEBANK","^CNXIT","^CNXAUTO","^CNXPHARMA","^CNXMETAL","^CNXREALTY","^CNXMEDIA","^CNXINFRA","NIFTY_OIL_AND_GAS.NS","NIFTY_HEALTHCARE.NS"]

In [10]:
## implementing a news based strategy


In [ ]:
def calculate_RSI(prices, period=14):
    # Calculate RSI here
    return rsi_values

def filter_Volume_Spike(volume, avg_volume, threshold=1.5):
    # Check for volume spikes here
    return volume > threshold * avg_volume

def filter_Uptrend_Downtrend(prices, ma_short_period=50, ma_long_period=200, rsi_period=14, rsi_threshold=50):
    # Calculate moving averages
    ma_short = calculate_moving_average(prices, ma_short_period)
    ma_long = calculate_moving_average(prices, ma_long_period)

    # Calculate RSI
    rsi_values = calculate_RSI(prices, rsi_period)

    # Check for uptrend or downtrend
    if ma_short[-1] > ma_long[-1] and rsi_values[-1] > rsi_threshold:
        return "Uptrend"
    elif ma_short[-1] < ma_long[-1] and rsi_values[-1] < (100 - rsi_threshold):
        return "Downtrend"
    else:
        return "Neutral"

def filter_reversals(prices, volume, avg_volume, ma_short_period=50, ma_long_period=200, rsi_period=14, rsi_threshold=50, volume_threshold=1.5):
    trend = filter_Uptrend_Downtrend(prices, ma_short_period, ma_long_period, rsi_period, rsi_threshold)

    if trend == "Neutral":
        return False

    volume_spike = filter_Volume_Spike(volume, avg_volume, volume_threshold)
    price_against_trend = (trend == "Uptrend" and prices[-1] < ma_short[-1]) or (trend == "Downtrend" and prices[-1] > ma_short[-1])

    return volume_spike and price_against_trend

# Main part of the strategy
def main():
    # Get historical price data and volume data for a stock
    prices = get_historical_prices('AAPL')
    volume = get_volume_data('AAPL')

    # Calculate moving averages and RSI
    ma_short_period = 50
    ma_long_period = 200
    rsi_period = 14
    rsi_threshold = 50
    ma_short = calculate_moving_average(prices, ma_short_period)
    ma_long = calculate_moving_average(prices, ma_long_period)
    rsi_values = calculate_RSI(prices, rsi_period)

    # Calculate average volume over a specified period (e.g., 30 days)
    avg_volume_period = 30
    avg_volume = calculate_average_volume(volume, avg_volume_period)

    # Filter out potential reversals
    if filter_reversals(prices, volume, avg_volume, ma_short_period, ma_long_period, rsi_period, rsi_threshold, volume_threshold=1.5):
        print("Potential reversal detected.")
    else:
        print("No potential reversal detected.")
